# Introduction

|Key|Value|
|:--|:--|
|Author|Firas Midani|
|E-mail|firas.midani@duke.edu|
|Last modified on|2018-02-15|

In this notebook, I provide all of the necessary code to reproduce supervised classification in Midani et *al*. (2018) paper titled "Human gut microbiota predicts suceptibility to *V. cholerae* infection". In particular, you will be able to run several distinct models by changing only a couple of [model parameters](#Define-model-parameters) for each analysis. In particular, you have the following options:

Classification of cholera susceptibility using 
 * known clinical/epidemological risk factors.
 * relative abundance of OTUs.
 * presence or absence of OTUs.
 * both clinical risk factors and relative abundance of OTUs.

In this notebook, you can also apply any of these classifications under different schemes: 
* No cross-validation (`cv = "NONE"`)
* Hold-out validation (`cv = "HOV"`)
* Stratified 10-fold cross-validation (`cv = "SKF"`)

***

*For running clinical model, use the following parameters<br/>*
* `include_microbe = False`
* `include_clinical = True`

*For running microbiota model, use the following parameters<br/>*
* `include_microbe = True`
* `include_clinical = False`

*For running combined model, use the following parameters<br/>*
* `include_microbe = True`
* `include_clinical = True`

*For running model on presence or absence of OTUs, use the following parameters<br/>*
* `include_microbe = True`
* `include_clinical = True`
* `binarize_microbes = True`

*** 

In Midani et al. (2018), we focus our analysis on associating microbiomes with cholera susceptibility. In an alternative model, we also run our model on a subset of individuals who are at the early stages of infection. In particular, these individuals were initailly excldued in the study, but later excluded to 16S rRNA gene signatures of *V. cholerae* at their baseline rectal swabs collected at enrollment (day 2). To run this alternative model, change `model_type` form `"susceptibility"` to `"onset"`


*** 

Notes: Data generated by this notebook may vary slighlty from data/numbers included in the corresponding published article because of different random number seeds. Further, due to high memory requirements, permutation testing (for statistical significance) on these models were performed on HARDAC (High-throughput Applied Research Data Analysis Cluster) which is partially supported by grant 2016-IDG-1013 from the NorthCarolina Biotechnology Center to Duke Center for Genomics and Computational Biology (GCB). This code was modified to communciate with the Slurm Workload Manger accordingly and it is hosted on [Github](https://github.com/LAD-LAB/supervised-classification).

# Import necessary packages

In [ ]:
import os
import sys
import imp
import time 
import random
import datetime

import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.feature_selection  import RFE
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, binarize
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.svm import SVC

print 'pandas\t\t%s' % pd.__version__
print 'numpy\t\t%s' % np.__version__
print 'scikit-learn\t%s' % sk.__version__
print 'seaborn\t\t%s' % sns.__version__
print 'matplotlib\t%s' % mpl.__version__
print 

RandomSeed = int(np.random.randint(1,10**9))
np.random.seed(RandomSeed)
print 'random = %s' % RandomSeed

sns.set_style('whitegrid')
%matplotlib inline

# Define user-made functions

In [ ]:
def txt_to_df(filename):
    return pd.read_csv(filename,sep='\t',header=0,index_col=0)

def binarizeMicrobes(df):

    binary_df = pd.DataFrame(binarize(df),index=df.index,columns=df.keys());

    return binary_df

def removeRareFeatures(df,threshold=0.10):
    
    '''
    remove features present in less thant percent (threshold) of the samples
    '''
    
    # binarize feature matrix  
    binary_df = pd.DataFrame(binarize(df),index=df.index,columns=df.keys());
        
    # identify prevalent features based on minimum cut-off
    minCount = np.ceil(threshold*df.shape[0]);
    prevalentFeatures = np.where(binary_df.apply(np.sum)>=minCount)[0];    
    prevalentFeatures = df.keys()[prevalentFeatures];
    
    # reduce feature matrix to prevalent OTUs
    df = df.loc[:,prevalentFeatures]
    
    return df

def trasnformFeatureMatrix(df,transformer):
    '''
    transform all values in dataframe
    '''
    
    return df.apply(transformer)

def scaleFeatureMatrix(df,scaler):
    '''
    scale each feature array in a dataframe
    '''
    
    if isinstance(df,pd.DataFrame):
        df = [df];
    
    df_scale_fit = scaler.fit(df[0]); 
    
    for ii in range(len(df)):
        df[ii] = pd.DataFrame(df_scale_fit.transform(df[ii]),
                             index=df[ii].index,
                             columns=df[ii].keys());
    
    if len(df)>1:
        return df[0],df[1]
    else:
        return df[0]

    
def initializeMetricTracker(num_folds,num_resamples):
    '''
    initialize pandas.DataFrame for tracking area model performance metric (i.e. AuROC)
    '''

    array_1 = sorted([ii+1 for ii in range(num_resamples)]*num_folds)
    array_2 = [ii+1 for ii in range(num_folds)]*num_resamples
    arrays = [array_1,array_2]
    tuples = list(zip(*arrays))

    index = pd.MultiIndex.from_tuples(tuples,names=['Sample','Fold'])

    df = pd.DataFrame(columns=index);
    df.index.name = 'Rank'

    return df

def initializeDecTracker(y_all,include_microbes,rfe_n_features_coarse):
    '''
    initialize pandas.DataFrame for tracking model decision scores on testing set
    '''
    if include_microbes:
        array = ['%s_f' % ii for ii in range(1,rfe_n_features_coarse+1)]
    else: 
        array = '1_f'
    df = pd.DataFrame(index=y_all.index,columns=[array])
    
    return df

def initializeCoefTracker(x_all):
    '''
    initialize pandas.DataFrame for tracking model feature coefficients
    '''

    df_1 = pd.DataFrame(index=x_all.keys())
    df_2 = pd.DataFrame(index=['vbxbase','ageyrs','bloodo'])
    df = pd.concat([df_1,df_2])
    df.index.name = 'coefficient'

    return df

def initializeFeatureTable():
    '''
    initialize pandas.DataFrame for features and their ranks
    '''

    df = pd.DataFrame(columns=['Feature'])
    df.index.name = 'Rank'
    
    return df


def crossValidationSplit(x_all,y_all,cv,num_folds,RandomSeed):
    
    '''
    define training and testing sets for all folds defined by cv argument
    '''

    if cv=='HOV':

        split_df = y_all.copy()
        split_df.loc[:,'color'] = [1 if int(ii.split('.')[1])<=42 else 2 for ii in split_df.index]
        
        train_index = [x_all.index.get_loc(ii) for ii in split_df[split_df.color==1].index]
        test_index = [x_all.index.get_loc(ii) for ii in split_df[split_df.color==2].index]
        indices = [(np.array(train_index),np.array(test_index))]
        return indices
    
    elif cv=='SKF':
        
        SKF = StratifiedKFold(n_splits=num_folds,shuffle=True,random_state=np.random.randint(1,10**9))
        return SKF.split(x_all,np.ravel(y_all));
    
    elif cv=='NONE':
        train_index = range(x_all.shape[0]);
        test_index = range(x_all.shape[0]);
        indices = [(np.array(train_index),np.array(test_index))]
        return indices
    
def extractMaximumPerformance(auc_df):
    ''' 
    in a pandas.DataFrame of single column, find maximum value and its corresponding index
    '''
    
    auc_max = auc_df.max()
    auc_index = auc_df.iloc[np.where(auc_df == auc_max)[0]].index[0]
    
    return auc_max, auc_index

def recordPrediction(parent_path,out_df,suffix):
    '''
    record pandas.DataFrame as CSV file under specific name
    '''

    if not os.path.exists(parent_path):
        os.makedirs(parent_path)
    
    filename = "%s/%s.txt" % (parent_path,suffix);
    out_df.to_csv(filename,sep='\t',header=True,index=True)
    
def generateUniqueFolderName(argsin):
    
    model_type,include_microbes,include_clinical,binarize_microbes,cv,num_folds,num_resamples,RandomSeed = argsin
    
    cv_map = {'NONE':'0','HOV':'1','SKF':2}
    
    folderName = str(["S" if model_type=='susceptibility' else "O"][0])
    folderName += str([1 if include_microbes else 0][0])
    folderName += str([1 if binarize_microbes else 0][0])
    folderName += str([1 if include_clinical else 0][0])
    folderName += "_cv%s_" % cv_map[cv]
    folderName += str(["%s_%s_" % (num_resamples,num_folds) if cv=="SKF" else ''][0])
    folderName += str(RandomSeed)
    
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    
    log = open('jobs_log.txt','a+');
    log.write('%s\t%s\n' % (st,folderName))

    return folderName

# Define model parameters

In [ ]:
## save output
save_output = False;

## type of model
model_type = 'onset' # susceptibility or onset
include_microbes = False;
include_clinical = True;

## use relative abundance (True) or presence/absence only (False)
binarize_microbes = False;

## feature filtering parameters
minPrevalence = 0.10;

## feature transformation parameters
pseudocount = 1e-6
transformer = lambda x: np.log10(x+pseudocount);
transformVbx = lambda x: np.log10(x+1);
binarizeAge = lambda x: 1 if x>=10 else 0;

## feature scaling parameters
scaler = StandardScaler()

## recursive feature elimination parameters
rfe_n_features_coarse = 501;
rfe_n_features_fine = 1;
rfe_step_coarse = 10;
rfe_step_fine = 1;

## supervised classification
MicrobialClassifier = SVC(kernel = 'linear', C = 100, 
                          probability = True, shrinking = True,
                          cache_size = 2000, random_state = RandomSeed)
ClinicalClassifier = LogisticRegression(penalty='l2',C=100,random_state = RandomSeed)

## coarse recursive feature elimination
CoarseRFE = RFE(estimator = MicrobialClassifier,
                n_features_to_select = rfe_n_features_coarse,
                step = rfe_step_coarse);

## cross-validation scheme
cv = 'SKF'

## k-fold cross-validation parameters (do not applly for HOV or NONE)
num_folds = 10 
num_resamples = 30

argsin = [model_type,include_microbes,include_clinical,binarize_microbes,cv,num_folds,num_resamples,RandomSeed]

## model output 
parent_path = './output_data/%s'% generateUniqueFolderName(argsin); print parent_path

# Import data

In [ ]:
if model_type == 'susceptibility':
    
    txt_y_all = './input_data/suscpetibility/outcomes.ygbr.day.2.txt';
    txt_x_all = './input_data/suscpetibility/otus.ygbr.day.2.txt';
    txt_c_all = './input_data/suscpetibility/clinical.ygbr.day.2.txt';

elif model_type == 'onset':
    
    txt_y_all = './input_data/onset/outcomes.infected.batch.1.day.2.txt';
    txt_x_all = './input_data/onset/features.S.no.vc.infected.day.2.txt';
    txt_c_all = './input_data/onset/clinical.infected.batch.1.day.2.txt';

c_all = txt_to_df(txt_c_all);
y_all = txt_to_df(txt_y_all);
x_all = txt_to_df(txt_x_all);

if binarize_microbes==True:
    x_all = binarizeMicrobes(x_all)

# log-10 transform vbxbase and binarize age based on >=10 y/o threshold
c_all.vbxbase = c_all.vbxbase.apply(transformVbx)
c_all.ageyrs = c_all.ageyrs.apply(binarizeAge)

# make sure data frames are properly ordered
x_all = x_all.loc[y_all.index,:]
c_all = c_all.loc[y_all.index,:]

print c_all.shape
print y_all.shape
print x_all.shape

# Run model and save results

In [ ]:
auc_shuffle = [];

for ii in range(100):
    
    print 'permutation #%s' % (ii+1),

    np.random.shuffle(y_all.values)

    # no need for multiple model instances if not cross-validating
    if (cv=="NONE") or (cv=="HOV"):
        num_resamples=1;
        num_folds=1;

    # initialize tables
    auc_df = initializeMetricTracker(num_folds,num_resamples);
    dec_df = initializeDecTracker(y_all,include_microbes,rfe_n_features_coarse);
    coef_df = initializeCoefTracker(x_all)
    features_df = initializeFeatureTable()

    # iterate through model instances
    for resample_number in range(num_resamples):

        #print
        #print 'model instance #%s' % (resample_number+1),

        # split training/testing sets
        cv_splits = crossValidationSplit(x_all,y_all,cv,num_folds,RandomSeed)

        # iterate through testing folds
        for fold_number,values in enumerate(cv_splits):

            #print '.', 
            # unpack iterator
            train_index,test_index = values

            # split training and testing 
            y_train = y_all.iloc[train_index]
            y_test = y_all.iloc[test_index]

            if include_clinical == True: 

                # split training and testing 
                C_train = c_all.iloc[train_index,:]
                C_test = c_all.iloc[test_index,:]

                # scale feature values to standard normal (into z-scores)
                C_train,C_test = scaleFeatureMatrix([C_train,C_test],scaler)

            if include_microbes == False:

                ## evaluate model performance on selected features
                CLF_fit = ClinicalClassifier.fit(C_train,np.ravel(y_train));
                CLF_dec = CLF_fit.decision_function(C_test);
                CLF_prb = CLF_fit.predict_log_proba(C_test)[:,1];
                CLF_auc = roc_auc_score(np.ravel(y_test),CLF_dec);

                ## record model performance
                auc_df.loc[1,(resample_number+1,fold_number+1)] = CLF_auc;

                if (cv=="NONE") or (cv=="HOV"):
                    dec_df.loc[y_test.index,'1_f'] = CLF_dec

            elif include_microbes == True:

                # split training and testing 
                X_train = x_all.iloc[train_index,:]
                X_test = x_all.iloc[test_index,:]

                # remove rare features
                X_train = removeRareFeatures(X_train,minPrevalence)
                X_test = X_test.loc[:,X_train.keys()]

                # logarithmic transform with pseudocount
                X_train = trasnformFeatureMatrix(X_train,transformer)
                X_test = trasnformFeatureMatrix(X_test,transformer)

                # scale feature values to standard normal (into z-scores)
                X_train,X_test = scaleFeatureMatrix([X_train,X_test],scaler)

                ##############################
                ## first stage of coarse rfe
                ##############################

                RFE1_fit = CoarseRFE.fit(X_train,np.ravel(y_train))
                RFE1_support = RFE1_fit.support_

                X_train_rfe1_support = X_train.keys()[RFE1_support];
                X_train_rfe1 = X_train.loc[:,X_train_rfe1_support];
                X_test_rfe1 = X_test.loc[:,X_train_rfe1_support];

                ## first stage of coarse rfe
                RFE1_fit = CoarseRFE.fit(X_train,np.ravel(y_train))
                RFE1_support = RFE1_fit.support_

                X_train_rfe1_support = X_train.keys()[RFE1_support];
                X_train_rfe1 = X_train.loc[:,X_train_rfe1_support];
                X_test_rfe1 = X_test.loc[:,X_train_rfe1_support];

                ##############################
                ## second stage of fine rfe
                ##############################

                # initialize feature matrix
                X_train_sfe = X_train_rfe1
                X_test_sfe = X_test_rfe1

                # feature rank from high to low
                features_rank = range(X_train_sfe.shape[1])[::-1]#[:-1]

                ## in descending fashion
                for ii in features_rank:

                    # if combiend model, join clinical and microbiota features
                    if include_clinical == True:
                        X_use_train = X_train_sfe.join(C_train);
                        X_use_test = X_test_sfe.join(C_test)
                    else:    
                        X_use_train = X_train_sfe;
                        X_use_test = X_test_sfe;

                    ## evaluate model performance on selected features
                    CLF_fit = MicrobialClassifier.fit(X_use_train,np.ravel(y_train));
                    CLF_dec = CLF_fit.decision_function(X_use_test);
                    CLF_auc = roc_auc_score(np.ravel(y_test),CLF_dec);

                    if (cv=="NONE") or (cv=="HOV"):
                        dec_df.loc[y_test.index,'%s_f' % (ii+1) ] = CLF_dec

                    if (cv=="NONE"):
                        coef = CLF_fit.coef_[0];
                        coef_df.loc[X_use_train.keys(),'%s_f' % (ii+1) ] = coef

                    ## record model performance
                    auc_df.loc[ii+1,(resample_number+1,fold_number+1)] = CLF_auc;

                    ## eliminate next feature
                    if X_train_sfe.shape[1]>1: 
                        SingleRFE = RFE(estimator = MicrobialClassifier,n_features_to_select=ii,step = 1)
                        SFE_fit = SingleRFE.fit(X_train_sfe,np.ravel(y_train))
                        support = SFE_fit.support_
                    else: 
                        support = np.array([False]);

                    ## which feature was removed 
                    FeaturesKept = X_train_sfe.keys()[support];
                    FeatureRemoved = X_train_sfe.keys()[~support].values[0];

                    ## reduce feature matrix 
                    X_train_sfe = X_train_sfe.loc[:,FeaturesKept];
                    X_test_sfe = X_test_sfe.loc[:,FeaturesKept];

                    ## record removed feature 
                    if cv=='NONE':
                        features_df.loc[ii+1,'Feature'] = FeatureRemoved

    ##############################
    ## recording model output 
    ##############################

    if save_output == True:

        recordPrediction(parent_path,auc_df,'auc_scores')

        if (cv=="NONE") or (cv=="HOV"):
            recordPrediction(parent_path,dec_df,'decision_scores')

        if (cv=="NONE") and (include_microbes==True):
            recordPrediction(parent_path,coef_df[coef_df.any(1)],'coefficients')
            recordPrediction(parent_path,features_df,'features')    

    #print
    #print 'Done!'
    
    print auc_df.mean(1).values[0]
    auc_shuffle.append(auc_df.mean(1).values[0])

In [ ]:
auc_df.mean(1).values[0]

# Summarize results

In [ ]:
# plt.hist(np.ravel([auc_df[ii].mean(1).values for ii in range(1,31)]));
# np.median(np.ravel([auc_df[ii].mean(1).values for ii in range(1,31)]))

## compute maximum AuROC (Area under the ROC curve)

In [ ]:
auc_df = auc_df[(~auc_df.isnull()).all(1)]
    
print 
print 'cv = %s' % cv
print 
print 'include_microbes = %s' % include_microbes
print 'binarize_microbes = %s' % binarize_microbes
print 'include_clinical = %s' % include_clinical
print 

auc_max, auc_index = extractMaximumPerformance(auc_df.mean(1))
print 'Maximum AUC = %0.4f using %i OTUs' % (auc_max, auc_index)


## plot AuROC against number of OTUs (if applicable)

In [ ]:
if include_microbes==True:
    fig,ax = plt.subplots(figsize=[5,5])
    ax.plot(auc_df.mean(1),lw=4,color=(0.,0.,0.,1.))

    ax.set_xlabel('Number of OTUs in model',fontsize=16)
    ax.set_ylabel('Area under the ROC curve (AuROC)',fontsize=16)

    ax.set_ylim([0.48,1.02])
    
    # automatically set padding on x-axis based on number of features coarsely selected
    if (auc_df.shape[0]%50) < 15:
        pad = 0
    else:
        pad = 1
        
    xlim_0 = -15
    xlim_1 = 50*((auc_df.shape[0]/50)+pad)+15
    
    ax.set_xlim([xlim_0,xlim_1])

    [ii.set(fontsize=16) for ii in ax.get_xticklabels()+ax.get_yticklabels()];
    
print 
print 'cv = %s' % cv
print 
print 'include_microbes = %s' % include_microbes
print 'binarize_microbes = %s' % binarize_microbes
print 'include_clinical = %s' % include_clinical
print 

auc_max, auc_index = extractMaximumPerformance(auc_df.mean(1))
print 'Maximum AUC = %0.4f using %i OTUs' % (auc_max, auc_index)


## plot ROC curve for optimal model (if applicable)

In [ ]:
if (cv=="HOV"):
    
    decision_scores = dec_df[dec_df.any(1)]    
    decision_scores = decision_scores.iloc[:,(auc_index+0)]
    true_labels = y_all.loc[decision_scores.index,:]
    tpr,fpr,thresholds = roc_curve(np.ravel(true_labels.values),decision_scores.values)

    fig,ax = plt.subplots(figsize=[5,5])
    
    ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='gray',
           label='Random guess', alpha=.8)
    ax.plot([0]+list(tpr),[0]+list(fpr),lw=8,color='k')

    ax.set_xlabel('False Positive Rate',fontsize=16)
    ax.set_ylabel('True Positive Rate',fontsize=16)

    ax.set_ylim([-0.05,1.05])
    ax.set_xlim([-0.05,1.05])

    [ii.set(fontsize=16) for ii in ax.get_xticklabels()+ax.get_yticklabels()];
    
print 
print 'cv = %s' % cv
print 
print 'include_microbes = %s' % include_microbes
print 'binarize_microbes = %s' % binarize_microbes
print 'include_clinical = %s' % include_clinical
print 

auc_max, auc_index = extractMaximumPerformance(auc_df.mean(1))
print 'Maximum AUC = %0.4f using %i OTUs' % (auc_max, auc_index)
